In [58]:
import os
import random

import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from IPython.display import Image
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers.legacy import Adam

"""WILL BE ADDED TO A SEPARATE MODULE LATER"""

def count_files(directory):
    count = 0
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Check if the path is a file (not a directory)
        if os.path.isfile(os.path.join(directory, filename)):
            count += 1
    return count

def get_image_label(filename):
    label = filename.split(".")[0].split("_")[0]
    return label

def load_images_from_dataset(directory):
    images = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            # Attempt to read the image
            image = cv2.imread(file_path)
            # Check if the image is not None
            if image is not None:
                # Convert the image to RGB format
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                label = get_image_label(filename)
                image_and_label = (image, label)
                images.append(image_and_label)
            else:
                print(f"Unable to read image file: {file_path}")
        except Exception as e:
            print(f"Error reading image file {file_path}: {str(e)}")
    return images
    

In [59]:
""" LOAD IMAGES FROM DATASET"""
current_dir = os.path.dirname(os.path.abspath("main.py"))
parent_dir = os.path.dirname(current_dir)
dataset_dir = os.path.join(parent_dir, "dataset")
train_dir = os.path.join(dataset_dir, "training")
test_dir = os.path.join(dataset_dir, "test")

# Training Data Dirs
proclined_training_dir = train_dir + "/proclined"
retroclined_training_dir = train_dir + "/retroclined"

proclined_training_images = load_images_from_dataset(proclined_training_dir)
retroclined_training_images = load_images_from_dataset(retroclined_training_dir)

merged_training_images = proclined_training_images + retroclined_training_images
random.shuffle(merged_training_images)

Unable to read image file: /Users/eaidy/Repos/ML/inclination-classification/dataset/training/proclined/.DS_Store
Unable to read image file: /Users/eaidy/Repos/ML/inclination-classification/dataset/training/retroclined/.DS_Store


In [60]:
""" PRE-PROCESSING LAYER """

# RESIZE IMAGES TO A STANDARD DIMENSION and EXTRACT LABELS TO ANOTHER ARRAY
""" When resizing, aspect ratio should be protected. Leaving it unconcerned now, will be handled later - ?"""
resized_training_images = []
labels = []
for image, label in merged_training_images:
    image = cv2.resize(image, (224, 224))
    if label == "proclined":
        labels.append(0)
    else:
        labels.append(1)
    resized_training_images.append(image)

# NUMPY ARRAY TRANSFORM
merged_training_images = np.array(resized_training_images)

# NORMALIZE IMAGES TO FLOATING POINT NUMBERS BETWEEN 0 and 1
merged_training_images = merged_training_images / 255.0

# DATA AUGMENTATION
datagen = ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True
)

final_training_data = datagen.flow(merged_training_images, labels, batch_size=32)

In [ ]:
""" CREATING THE MODEL AND TRAINING DATA """

input_shape = (224, 224, 3)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Change the output units based on your classification task
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(final_training_data, epochs=60, steps_per_epoch=len(merged_training_images) // 32)

In [62]:
test_datagen = ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True
)

test_data_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=4,
    class_mode='binary',
    shuffle=False  # No need to shuffle test data
)

steps_per_epoch = len(os.listdir(test_dir)) - 1

test_loss, test_acc = model.evaluate(test_data_generator, steps=(steps_per_epoch) // 32)
print('Test accuracy:', test_acc)

Found 0 images belonging to 0 classes.


ValueError: Unexpected value for `steps_per_epoch`. Received value is 0. Please check the docstring for `model.fit()` for supported values.